In [1]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os
from keras.models import Model
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout, Input, AveragePooling2D, Lambda
from urllib import request
from keras.callbacks import TensorBoard
import tempfile
from keras import optimizers
from keras.utils import Sequence
import numpy as np
from tqdm import tqdm
from datetime import datetime
from keras.applications import vgg16, resnet50

Using TensorFlow backend.


In [2]:
NOT_KAGGLE_KERNEL = os.environ.get('NOT_KAGGLE_KERNEL', 'false') == 'true'
print('Run on kaggle kernel:', NOT_KAGGLE_KERNEL)

Run on kaggle kernel: True


In [3]:
ROOT_DIR = '../input/dog-breed' if NOT_KAGGLE_KERNEL else '../input'
EPOCHS = 100
IMAGE_SIZE = (224,224)
INPUT_SHAPE = IMAGE_SIZE + (3,)

In [4]:
def fetch_data():
    if not os.path.exists(ROOT_DIR):
        os.makedirs(ROOT_DIR)
        zip_path = os.path.join(tempfile.gettempdir(), 'dog-breed.zip')
        print('Start download!')
        request.urlretrieve('https://kienle.blob.core.windows.net/public/kaggle/dog-breed.zip', zip_path)
        print('Start unzip')
        import zipfile
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(ROOT_DIR)
        with zipfile.ZipFile(os.path.join(ROOT_DIR,'test.zip'), 'r') as zip_ref:
            zip_ref.extractall(ROOT_DIR)
        with zipfile.ZipFile(os.path.join(ROOT_DIR,'train.zip'), 'r') as zip_ref:
            zip_ref.extractall(ROOT_DIR)
        os.remove(zip_path)
        os.remove(os.path.join(ROOT_DIR,'train.zip'))
        os.remove(os.path.join(ROOT_DIR,'test.zip'))
        print('Done')
if(NOT_KAGGLE_KERNEL):
    fetch_data()

In [5]:
# Confirm all data are ready 
# Expected output: labels.csv  sample_submission.csv  test  train
!ls $ROOT_DIR

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
pd_images = pd.read_csv(os.path.join(ROOT_DIR, 'labels.csv'))

In [7]:
pd_images.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [8]:
train_valid_generator = ImageDataGenerator().flow_from_dataframe(pd_images, os.path.join(ROOT_DIR, 'train'), batch_size=32, x_col='id', y_col='breed', has_ext=False, target_size=IMAGE_SIZE)

Found 10222 images belonging to 120 classes.


In [9]:
num_classes = len(train_valid_generator.class_indices)

In [10]:
class FeaturesExtractor():
    def __init__(self, input_shape, pretrained_model, preprocess_input):
        self.input_shape = input_shape
        model = pretrained_model(include_top=False, input_shape=self.input_shape, weights='imagenet')
        inputs = Input(self.input_shape)
        x = inputs
        x = Lambda(preprocess_input, name='preprocessing')(x)
        x = model(x)
        x = GlobalAveragePooling2D()(x)
        model = Model(inputs, x)
        self.model = model
    def predict(self, X):
        return self.model.predict(X)

In [11]:
vgg_features_extractor = FeaturesExtractor(INPUT_SHAPE, vgg16.VGG16, vgg16.preprocess_input)
resnet50_features_extractor = FeaturesExtractor(INPUT_SHAPE, resnet50.ResNet50, resnet50.preprocess_input)

C:\Users\trule\AppData\Local\Continuum\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [12]:
def create_simple_model(num_classes, input_shape):
    inputs = Input(input_shape)
    x = inputs
    x = Dropout(0.5)(x)
    x = Dense(num_classes, activation='softmax')(x)
    return Model(inputs, x)


In [13]:
X_train = np.zeros((train_valid_generator.n,) + INPUT_SHAPE)
y_train = np.zeros((train_valid_generator.n, 120))
for i in tqdm(range(len(train_valid_generator)), ncols=100):
    temp_x, temp_y = train_valid_generator[i]
    start_index = i*train_valid_generator.batch_size
    X_train[start_index:start_index + temp_x.shape[0]] = temp_x
    y_train[start_index:start_index + temp_x.shape[0]] = temp_y


100%|█████████████████████████████████████████████████████████████| 320/320 [00:43<00:00,  7.42it/s]


In [15]:
# Calculate vgg features
vgg_features = vgg_features_extractor.predict(X_train)

In [22]:
vgg_based_model = create_simple_model(num_classes, vgg_features_extractor.model.output.get_shape().as_list()[1:])
vgg_based_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
if NOT_KAGGLE_KERNEL:
    callbacks = [TensorBoard('./data/logs/vgg-{0}'.format(datetime.now().isoformat().replace(':','-').split('.')[0]))]
else:
    callbacks = []
    
vgg_based_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               61560     
Total params: 61,560
Trainable params: 61,560
Non-trainable params: 0
_________________________________________________________________


In [23]:
vgg_based_model.fit(vgg_features, y_train, batch_size=256, epochs=EPOCHS, callbacks=callbacks, validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/100
9199/9199 [==============================] - ETA: 7s - loss: 15.1982 - acc: 0.00 - ETA: 0s - loss: 15.1469 - acc: 0.00 - ETA: 0s - loss: 14.9867 - acc: 0.01 - ETA: 0s - loss: 14.8529 - acc: 0.01 - 1s 61us/step - loss: 14.7239 - acc: 0.0227 - val_loss: 12.2939 - val_acc: 0.0782
Epoch 2/100
9199/9199 [==============================] - ETA: 0s - loss: 14.1947 - acc: 0.03 - ETA: 0s - loss: 14.1402 - acc: 0.03 - ETA: 0s - loss: 13.9014 - acc: 0.04 - ETA: 0s - loss: 13.7647 - acc: 0.04 - 0s 23us/step - loss: 13.6788 - acc: 0.0505 - val_loss: 10.4772 - val_acc: 0.1486
Epoch 3/100
9199/9199 [==============================] - ETA: 0s - loss: 12.3968 - acc: 0.10 - ETA: 0s - loss: 12.7330 - acc: 0.08 - ETA: 0s - loss: 12.5942 - acc: 0.09 - ETA: 0s - loss: 12.3635 - acc: 0.09 - 0s 21us/step - loss: 12.3028 - acc: 0.0960 - val_loss: 8.4251 - val_acc: 0.2258
Epoch 4/100
9199/9199 [==============================] - ETA: 0s - loss: 11.3865 - 

9199/9199 [==============================] - ETA: 0s - loss: 2.7909 - acc: 0.632 - ETA: 0s - loss: 3.0820 - acc: 0.627 - ETA: 0s - loss: 3.0570 - acc: 0.621 - ETA: 0s - loss: 3.0938 - acc: 0.618 - 0s 20us/step - loss: 3.0779 - acc: 0.6195 - val_loss: 3.1362 - val_acc: 0.6237
Epoch 31/100
9199/9199 [==============================] - ETA: 0s - loss: 3.0640 - acc: 0.589 - ETA: 0s - loss: 3.0543 - acc: 0.615 - ETA: 0s - loss: 3.0276 - acc: 0.619 - 0s 18us/step - loss: 3.0667 - acc: 0.6169 - val_loss: 3.1216 - val_acc: 0.6227
Epoch 32/100
9199/9199 [==============================] - ETA: 0s - loss: 3.0687 - acc: 0.648 - ETA: 0s - loss: 2.9339 - acc: 0.637 - ETA: 0s - loss: 2.9550 - acc: 0.629 - 0s 18us/step - loss: 2.9896 - acc: 0.6266 - val_loss: 3.0545 - val_acc: 0.6285
Epoch 33/100
9199/9199 [==============================] - ETA: 0s - loss: 2.6990 - acc: 0.636 - ETA: 0s - loss: 2.8583 - acc: 0.634 - ETA: 0s - loss: 2.8599 - acc: 0.638 - 0s 18us/step - loss: 2.9627 - acc: 0.6292 - val_lo

9199/9199 [==============================] - ETA: 0s - loss: 2.0614 - acc: 0.683 - ETA: 0s - loss: 2.2440 - acc: 0.685 - ETA: 0s - loss: 2.2629 - acc: 0.678 - 0s 18us/step - loss: 2.3308 - acc: 0.6724 - val_loss: 2.6341 - val_acc: 0.6354
Epoch 62/100
9199/9199 [==============================] - ETA: 0s - loss: 2.2643 - acc: 0.695 - ETA: 0s - loss: 2.2808 - acc: 0.678 - ETA: 0s - loss: 2.2833 - acc: 0.677 - 0s 18us/step - loss: 2.3169 - acc: 0.6721 - val_loss: 2.6589 - val_acc: 0.6344
Epoch 63/100
9199/9199 [==============================] - ETA: 0s - loss: 2.3399 - acc: 0.699 - ETA: 0s - loss: 2.2645 - acc: 0.682 - ETA: 0s - loss: 2.3611 - acc: 0.672 - ETA: 0s - loss: 2.3235 - acc: 0.677 - 0s 19us/step - loss: 2.3263 - acc: 0.6770 - val_loss: 2.6594 - val_acc: 0.6364
Epoch 64/100
9199/9199 [==============================] - ETA: 0s - loss: 2.0796 - acc: 0.707 - ETA: 0s - loss: 2.2042 - acc: 0.692 - ETA: 0s - loss: 2.2506 - acc: 0.682 - 0s 17us/step - loss: 2.3089 - acc: 0.6751 - val_lo

9199/9199 [==============================] - ETA: 0s - loss: 2.4521 - acc: 0.679 - ETA: 0s - loss: 2.1849 - acc: 0.690 - ETA: 0s - loss: 2.2339 - acc: 0.680 - 0s 18us/step - loss: 2.2352 - acc: 0.6795 - val_loss: 2.7022 - val_acc: 0.6315
Epoch 93/100
9199/9199 [==============================] - ETA: 0s - loss: 2.4643 - acc: 0.636 - ETA: 0s - loss: 2.1544 - acc: 0.684 - ETA: 0s - loss: 2.2636 - acc: 0.678 - 0s 18us/step - loss: 2.2372 - acc: 0.6772 - val_loss: 2.6316 - val_acc: 0.6383
Epoch 94/100
9199/9199 [==============================] - ETA: 0s - loss: 2.4090 - acc: 0.644 - ETA: 0s - loss: 2.2224 - acc: 0.684 - ETA: 0s - loss: 2.2350 - acc: 0.679 - 0s 18us/step - loss: 2.2364 - acc: 0.6780 - val_loss: 2.6584 - val_acc: 0.6354
Epoch 95/100
9199/9199 [==============================] - ETA: 0s - loss: 2.4846 - acc: 0.664 - ETA: 0s - loss: 2.1503 - acc: 0.694 - ETA: 0s - loss: 2.1932 - acc: 0.682 - 0s 18us/step - loss: 2.2367 - acc: 0.6791 - val_loss: 2.6118 - val_acc: 0.6237
Epoch 96/

In [19]:
resnet50_features = resnet50_features_extractor.predict(X_train)

In [24]:
resnet50_based_model = create_simple_model(num_classes, resnet50_features_extractor.model.output.get_shape().as_list()[1:])
resnet50_based_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
if NOT_KAGGLE_KERNEL:
    callbacks = [TensorBoard('./data/logs/resnet50-{0}'.format(datetime.now().isoformat().replace(':','-').split('.')[0]))]
else:
    callbacks = []

In [25]:
resnet50_based_model.fit(resnet50_features, y_train, batch_size=256, epochs=EPOCHS,  callbacks=callbacks, validation_split=0.1)

Train on 9199 samples, validate on 1023 samples
Epoch 1/100
9199/9199 [==============================] - ETA: 7s - loss: 6.1339 - acc: 0.007 - ETA: 0s - loss: 5.5648 - acc: 0.023 - ETA: 0s - loss: 4.9517 - acc: 0.059 - ETA: 0s - loss: 4.4822 - acc: 0.100 - ETA: 0s - loss: 4.0715 - acc: 0.152 - 1s 66us/step - loss: 3.9010 - acc: 0.1751 - val_loss: 1.7492 - val_acc: 0.5787
Epoch 2/100
9199/9199 [==============================] - ETA: 0s - loss: 2.2155 - acc: 0.457 - ETA: 0s - loss: 2.0251 - acc: 0.478 - ETA: 0s - loss: 1.8951 - acc: 0.502 - ETA: 0s - loss: 1.8119 - acc: 0.515 - ETA: 0s - loss: 1.7327 - acc: 0.534 - 0s 28us/step - loss: 1.7052 - acc: 0.5399 - val_loss: 1.1239 - val_acc: 0.6940
Epoch 3/100
9199/9199 [==============================] - ETA: 0s - loss: 1.5109 - acc: 0.593 - ETA: 0s - loss: 1.3013 - acc: 0.636 - ETA: 0s - loss: 1.2573 - acc: 0.646 - ETA: 0s - loss: 1.2225 - acc: 0.652 - ETA: 0s - loss: 1.1927 - acc: 0.660 - 0s 27us/step - loss: 1.1831 - acc: 0.6626 - val_loss:

Epoch 26/100
9199/9199 [==============================] - ETA: 0s - loss: 0.2022 - acc: 0.945 - ETA: 0s - loss: 0.1663 - acc: 0.960 - ETA: 0s - loss: 0.1628 - acc: 0.959 - ETA: 0s - loss: 0.1629 - acc: 0.959 - ETA: 0s - loss: 0.1611 - acc: 0.960 - 0s 27us/step - loss: 0.1620 - acc: 0.9595 - val_loss: 0.7380 - val_acc: 0.7761
Epoch 27/100
9199/9199 [==============================] - ETA: 0s - loss: 0.1562 - acc: 0.960 - ETA: 0s - loss: 0.1536 - acc: 0.967 - ETA: 0s - loss: 0.1623 - acc: 0.959 - ETA: 0s - loss: 0.1614 - acc: 0.960 - ETA: 0s - loss: 0.1623 - acc: 0.959 - 0s 28us/step - loss: 0.1622 - acc: 0.9596 - val_loss: 0.7555 - val_acc: 0.7654
Epoch 28/100
9199/9199 [==============================] - ETA: 0s - loss: 0.1313 - acc: 0.968 - ETA: 0s - loss: 0.1474 - acc: 0.962 - ETA: 0s - loss: 0.1451 - acc: 0.960 - ETA: 0s - loss: 0.1517 - acc: 0.958 - 0s 26us/step - loss: 0.1560 - acc: 0.9579 - val_loss: 0.7482 - val_acc: 0.7703
Epoch 29/100
9199/9199 [==============================] -

9199/9199 [==============================] - ETA: 0s - loss: 0.0558 - acc: 1.000 - ETA: 0s - loss: 0.0865 - acc: 0.979 - ETA: 0s - loss: 0.0853 - acc: 0.980 - ETA: 0s - loss: 0.0880 - acc: 0.979 - ETA: 0s - loss: 0.0896 - acc: 0.978 - 0s 27us/step - loss: 0.0897 - acc: 0.9786 - val_loss: 0.8196 - val_acc: 0.7634
Epoch 52/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0818 - acc: 0.980 - ETA: 0s - loss: 0.0846 - acc: 0.979 - ETA: 0s - loss: 0.0906 - acc: 0.976 - ETA: 0s - loss: 0.0914 - acc: 0.977 - 0s 26us/step - loss: 0.0918 - acc: 0.9766 - val_loss: 0.8009 - val_acc: 0.7722
Epoch 53/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0715 - acc: 0.988 - ETA: 0s - loss: 0.0813 - acc: 0.980 - ETA: 0s - loss: 0.0823 - acc: 0.978 - ETA: 0s - loss: 0.0842 - acc: 0.978 - ETA: 0s - loss: 0.0869 - acc: 0.978 - 0s 28us/step - loss: 0.0870 - acc: 0.9785 - val_loss: 0.8115 - val_acc: 0.7664
Epoch 54/100
9199/9199 [==============================] - ETA: 0s - lo

9199/9199 [==============================] - ETA: 0s - loss: 0.0979 - acc: 0.972 - ETA: 0s - loss: 0.0724 - acc: 0.979 - ETA: 0s - loss: 0.0713 - acc: 0.980 - ETA: 0s - loss: 0.0701 - acc: 0.982 - ETA: 0s - loss: 0.0719 - acc: 0.981 - 0s 27us/step - loss: 0.0713 - acc: 0.9818 - val_loss: 0.8525 - val_acc: 0.7634
Epoch 77/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0704 - acc: 0.984 - ETA: 0s - loss: 0.0632 - acc: 0.987 - ETA: 0s - loss: 0.0650 - acc: 0.985 - ETA: 0s - loss: 0.0674 - acc: 0.983 - ETA: 0s - loss: 0.0665 - acc: 0.983 - 0s 27us/step - loss: 0.0665 - acc: 0.9838 - val_loss: 0.8948 - val_acc: 0.7625
Epoch 78/100
9199/9199 [==============================] - ETA: 0s - loss: 0.0535 - acc: 0.988 - ETA: 0s - loss: 0.0593 - acc: 0.985 - ETA: 0s - loss: 0.0616 - acc: 0.984 - ETA: 0s - loss: 0.0626 - acc: 0.983 - ETA: 0s - loss: 0.0634 - acc: 0.983 - 0s 27us/step - loss: 0.0638 - acc: 0.9833 - val_loss: 0.8734 - val_acc: 0.7703
Epoch 79/100
9199/9199 [========